In [4]:
import os
import xml.etree.ElementTree as ET
import torch
import torchvision
from torchvision.transforms import functional as F
from PIL import Image

# dataset = CustomDataset(root="path/to/YourDatasetFolder", transforms=None)


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "Anotation"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "JPEGImages", self.imgs[idx])
        ann_path = os.path.join(self.root, "Annotations", self.annotations[idx])
        img = Image.open(img_path).convert("RGB")

        # Parse the XML file
        tree = ET.parse(ann_path)
        root = tree.getroot()
        boxes = []
        for obj in root.findall('object'):
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((len(boxes),), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target


In [17]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=5)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\baldo/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 11.4MB/s]


In [20]:
from torch.utils.data import DataLoader

import os
import xml.etree.ElementTree as ET
import torch
from torchvision.transforms import functional as F
from PIL import Image

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "Anotation"))))

    def __getitem__(self, idx):
        # Your __getitem__ implementation
        pass

    def __len__(self):
        return len(self.imgs)

# Now you can create a DataLoader without the TypeError
dataset = CustomDataset(root="Dataset", transforms=None)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0, pin_memory=False)

# Print a success message
print("DataLoader created successfully with CustomDataset.")


DataLoader created successfully with CustomDataset.


In [21]:
# Assuming you have a DataLoader for your dataset
for epoch in range(100):
    for images, targets in dataloader:
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [24]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# Define the number of classes in your dataset (including the background class)
num_classes = 5

# Load the pre-trained ResNet50-FPN backbone
backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone('resnet50', pretrained=True)

# Create the Faster R-CNN model
model = FasterRCNN(
    backbone,
    num_classes=num_classes,
    rpn_anchor_generator=AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
)

# Assuming you have a DataLoader for your dataset
for epoch in range(100):
    for images, targets in dataloader:
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'faster_rcnn_custom_model.pth')


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [25]:
# Debugging snippet for inspecting data in your CustomDataset
for images, targets in dataloader:
    # Print the shape of images and targets (you can customize this part)
    print(f"Batch shape - Images: {images.shape}, Targets: {targets}")

    # Inspect individual samples (you can customize this part)
    for i in range(len(images)):
        print(f"Sample {i + 1} - Image shape: {images[i].shape}, Target: {targets[i]}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [26]:
import torch

# Custom collate function to handle your specific data format
def custom_collate(batch):
    # Filter out any None values
    batch = [sample for sample in batch if sample is not None]
    return batch

# Assuming you have a DataLoader for your dataset
for epoch in range(100):
    for images, targets in dataloader:
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'faster_rcnn_custom_model.pth')


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>